In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
e

{'time': Timestamp('2025-03-12 09:31:00'),
 'exp_id': '8e82b9f8-95e1-4d2c-8526-dda1139fa307',
 'return_event_uuid': 'fd5cfa54-8a01-4eb5-8ec0-cfd3d4cd5295',
 'agent_uuid': '94dbfaa4-97a6-4a2d-b928-eccae4938419',
 'agent_uuid_codename': 'tasty-pitch',
 'time_spent_ride_lift': 6.071562547769156,
 'time_spent_traverse_down_mountain': 4.961262873092367,
 'time_spent_in_queue': 5,
 'time_spent_total': 16.032825420861524}

In [2]:
from zero_helpers.imports import * 
from zero_liftsim.simmanager import SimulationManager
from zero_liftsim.helpers import base_config
from zero_liftsim.experience import AgentRideLoopExperience
from zero_liftsim.helpers import load_asset_template
from zero_liftsim import helpers as hp
from zero_liftsim.lift import Lift

import inspect

# Run Simulation

In [3]:
cfg = base_config()
cfg["SimulationManager"]["__init__"].update({"n_agents": 3, "lift_capacity": 2})
manager = SimulationManager(cfg)
result = manager.run(runtime_minutes=60)
print(result)

{'total_rides': 12, 'average_wait': 2.75, 'agents': [Agent(1) tacit-user 1be840dc-1140-4947-b6f0-2dcc233c0023, Agent(2) seemly-tackle 780a20a8-3427-48df-8e02-ad3b9251f5ce, Agent(3) tangible-maintenance e65ea3cc-6c36-4bf0-9a61-93701546163e]}


In [4]:
lift = manager.lift
lift.total_chairs()

50

## Retrieve Data from Simulation

In [5]:
# get rideloop explogs and agent event log
exp_log_data = manager.retrieve_data()
exp = exp_log_data['exp_rideloop']
log = exp_log_data['agent_log']

# Dev

In [6]:
# sample agent and subset exp/log to e/l for agent subset of those
m = log['event'] == 'ride_complete'
event_ride_comp = log[m].sample().iloc[0].to_dict()
agent = manager.lookup_agent(event_ride_comp['agent_uuid'])
l = log[log['agent_uuid'] == agent.agent_uuid]
idx = (exp['time'] - event_ride_comp['time']).dt.total_seconds().abs().idxmin()
e = exp.loc[idx].to_dict()

In [7]:
x = agent.traceback_experience(e['exp_id'])
print(x)

# Experience Log 

exp_id: 24da8952-5247-45dd-a68c-8c9bdbecc30c
agent_uuid: e65ea3cc-6c36-4bf0-9a61-93701546163e
return_event_uuid: eb85fd4c-9d0f-4c9e-8a8a-7ebe12b896d8

*** 


time_spent_in_queue: 5
time_spent_ride_lift: 7.284298220247956
time_spent_traverse_down_mountain: 4.371494866232662

***

# Full Dict

{
    "exp_id": "24da8952-5247-45dd-a68c-8c9bdbecc30c",
    "return_event_uuid": "eb85fd4c-9d0f-4c9e-8a8a-7ebe12b896d8",
    "agent_uuid": "e65ea3cc-6c36-4bf0-9a61-93701546163e",
    "agent_uuid_codename": "tangible-maintenance",
    "time_spent_ride_lift": 7.284298220247956,
    "time_spent_traverse_down_mountain": 4.371494866232662,
    "time_spent_in_queue": 5,
    "time_spent_total": 16.655793086480617
}

# Agent Log

{
    "0941c6de-5d4f-4b2e-b296-e5f6ef2d5553": {
        "return_event_uuid": "",
        "time": "2025-03-12T09:02:00",
        "time_offset": 2,
        "event": "start_wait",
        "agent_id": 3,
        "agent_uuid": "e65ea3cc-6c36-4bf0-9a61-93701546163e",
